In [ ]:
import pytreebank
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import random
import re

In [ ]:
bank = pytreebank.load_sst("stanfordSentimentTreebank/trees")

dictFile = open("stanfordSentimentTreebank/dictionary.txt")
lines = dictFile.readlines()
exp = r'^(\S+)\|\d+$'
unfilterdWords = map(lambda line: re.findall(exp, line), lines)
words = list(map(lambda x: x.replace('\\', ''),
            map(lambda x: x[0],
            filter(lambda x: len(x) > 0, 
            unfilterdWords))))
words.append('8 1/2')
words.append('2 1/2')
words.append('9 1/2')

dictionary = dict((word, number) for number, word in enumerate(words))

def oneHotEncoding(word):
    ##if word.find('/') != -1:
    ##    tmp = word.split('/')
    ##    word = tmp[0] + '\/' + tmp[1]
    vector = torch.zeros(1, len(dictionary)).cuda()
    vector[0][dictionary[word]] = 1
    return vector

def getLabels(tree):
    labels = []
    if len(tree.children) == 2:
        labels.extend(getLabels(tree.children[0]))
        labels.extend(getLabels(tree.children[1]))
        labels.extend([tree.label])
        return labels
    else:
        return [tree.label]

In [ ]:
testNet = torch.load('./net.pth')
testNet.eval()

with torch.no_grad():
    correct = 0
    total = len(bank['test'])
    for sentence in bank['test']:
        outputs = testNet(sentence)
        targets = torch.tensor(getLabels(sentence))

        if torch.argmax(outputs[-1]) == targets[-1]:
            correct += 1

    print(f'Fine grained accuracy on the test set: {correct/total * 100}%')